**IL SECONDO TASK DA PARTE DEL CLIENTE ERA DI AVERE L'INFERENZA MENO LENTA**  
*PARTI DA RIVEDERE:*   
-UTILIZZARE SOLO MODELLI MENO GRANDI;
-ASSICURARMI CHE IL MAC STIA UTILIZZANDO LA GPU ('mps');  
-RIDUCO IL PROMPT TEMPLATE;  
-RIDURRE IL NUMERO DEI VALORI CHE PRENDO IN CONSIDERAZIONE CON L'EMBEDDING DELLA QUERY PER LA COLLECTION E CONSIDERO SOLO IL MIGLIOR RISULTATO (n_result=1)   
-INSERIRE UNO STREAM DEL PROCESSO IN MODO DA RENDERE DINAMICA L' ATTESA PER L'UTENTE E MENO 'MORTA'.


**PARTE 1 DEL PROGETTO**
***FACCIO IMPORT***

In [1]:
import os 
import fitz #to import pdf
import re
import chromadb #for vectordb
from FlagEmbedding import BGEM3FlagModel
import ollama
import time
import torch
import pandas as pd
from tqdm import tqdm

**MI ASSICURO DI UTILIZZARE LA GPU**

In [2]:
if torch.backends.mps.is_available():
    device = 'mps'
    print('mps ready')
else:       
    device = 'cpu'
    print('=( no mps')

mps ready


**CARICO IL MODELLO DI EMBEDDING CHE HO SCELTO**

In [3]:
print("Caricamento del modello BGE-M3...")
model_embedding = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True, device=device)
print("Modello caricato.")

Caricamento del modello BGE-M3...


Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

Modello caricato.


**SETTO I PERCORSI DEI FILE**

In [4]:
PDF_PATH = "ETR_500.pdf" 
DB_PATH = "/Users/raffaeleciccarone/Desktop/project_ibm/chroma_db_data"         
COLLECTION_NAME = "manuale_manutenzione_ETR500"

In [5]:
!ollama pull llama3
!ollama pull llama3.2
!ollama pull granite3-dense:2b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest 
pulling 6a0746a1ec1a: 100% ▕██████████████████▏ 4.7 GB                         
pulling 4fa551d4f938: 100% ▕██████████████████▏  12 KB                         
pulling 8ab4849b038c: 100% ▕██████████████████▏  254 B                         
pulling 577073ffcc6c: 100% ▕██████████████████▏  110 B                         
pulling 3f8eb4da87fa: 100% ▕██████████████████▏  485 B                         
verifying sha256 digest 
writing manifest 
success 
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB         

**TEST EFFICIENZA IN TEMPO**   
*testo 2 modelli*

In [6]:
df_risposte = pd.DataFrame(columns=['model', 'time', 'query', 'response'])

**TEST EFFICIENZA IN TEMPO**   
*testo 2 modelli*

In [7]:
query_text = ['Come effettuare la manutenzione ordinaria dei freni del treno ETR 500?',
              'quali sono le procedure per la sostituzione dei rubinetti di testa del treno ETR 500?',
              'come si esegue la prova freno a banco sul treno ETR 500?',
              'come si ispezionano le sale del treno ETR 500 durante la manutenzione ordinaria?',
              'quali controlli non distruttivi sono previsti per le ruote del treno ETR 500?']

model_list = ['llama3.2', 'granite3-dense:2b']

def final_test(query_text, model, DB_PATH, COLLECTION_NAME, model_list):
    global df_risposte
    print(f"Il tuo aiutante manutentore sta pensando: '{query_text}'...\n\nCIUUUUUF CIUUUUUUF!")

    #retrieve from vectordb
    query_vec = model.encode(query_text)['dense_vecs']

    client = chromadb.PersistentClient(path=DB_PATH)
    collection = client.get_or_create_collection(name=COLLECTION_NAME)
    results = collection.query(query_embeddings=query_vec, n_results=1)

    # texts in a single string
    context_list = results['documents'][0]
    context_text = '\n---\n'.join(context_list)

    #define model prompt with summary of all retrieved chunks plus user query
    prompt_template = f"""
    Sei esperto nella manutenzione ferroviaria del treno ETR 500. Basati ESCLUSIVAMENTE sul contesto fornito.
    NON inventare procedure. Se l'informazione non è nel contesto dì che non hai la risposta.

    CONTESTO ESTRATTO DAL MANUALE:
    {context_text}

    DOMANDA UTENTE:
    {query_text}

    RISPOSTA (in italiano tecnico e preciso):
    """
    for m in model_list:
        print(f"\n\n=== TEST DEL MODELLO: {m} ===")
        #GOOOOOOOOOOOOOOO!!!
        start_time = time.time()

        '''
        response = ollama.chat(model=m, messages=[
            {'role': 'user', 'content': prompt_template},
        ])
        '''
        
        stream = ollama.chat(model=m, messages=[
            {'role': 'user', 'content': prompt_template},
        ], stream=True)

        response = ''

        for piece in tqdm(stream, desc=f" {m} sta scrivendo...", unit=" token"):
            text_part = piece['message']['content']
            response += text_part

        #stop and print...
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f'Tempo per inferenza: {elapsed_time:.2f} secondi')

        new_row = {
            'model': m,
            'time': round(elapsed_time, 2),
            'query': query_text,   
            'response': response
        }
        
        # Aggiungiamo al DF
        df_risposte = pd.concat([df_risposte, pd.DataFrame([new_row])], ignore_index=True)

        print("\n" + "=D "*50)
        print(f"RISPOSTA GENERATA DA '{m}':")
        print("=P "*50)
        print(response)

for q in query_text:
    final_test(q, model_embedding, DB_PATH, COLLECTION_NAME, model_list)

Il tuo aiutante manutentore sta pensando: 'Come effettuare la manutenzione ordinaria dei freni del treno ETR 500?'...

CIUUUUUF CIUUUUUUF!


You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.




=== TEST DEL MODELLO: llama3.2 ===


 llama3.2 sta scrivendo...: 310 token [00:28, 10.97 token/s]
/var/folders/6j/pmjh74gd2rb3xdj3jp3fdyj00000gn/T/ipykernel_5874/3093624055.py:71: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_risposte = pd.concat([df_risposte, pd.DataFrame([new_row])], ignore_index=True)


Tempo per inferenza: 28.27 secondi

=D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D 
RISPOSTA GENERATA DA 'llama3.2':
=P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P 
Mi dispiace, ma non ho trovato alcuna informazione specifica sulla manutenzione ordinaria dei freni del treno ETR 500 nel contesto fornito. Il manuale di manutenzione esaminato è relativamente generale e non fornisce dettagli su procedure specifiche per la manutenzione dei freni.

Per quanto riguarda la manutenzione ordinaria dei freni, normalmente si verificherebbe:

- Controllare il livello del liquido di raffreddamento del freno.
- Verificare le condizioni delle pinze e dei freni in rubino.
- Controllare l'integrità delle linee guida del freno.
- Verificare che i freni siano funzionanti correttamente.

Tuttavia, non po

...

Tempo per inferenza: 32.47 secondi

=D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D 
RISPOSTA GENERATA DA 'granite3-dense:2b':
=P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P 
During the regular maintenance of the ETR 500 train, the cars are inspected as follows:

1. Exterior inspection: This includes checking for any physical damage, such as dents, scratches, or leaks. The exterior is also checked for proper mounting and securing of all components.
2. Interior inspection: The interior of the car is inspected for cleanliness, comfort, and safety. This includes checking for proper functioning of all amenities such as lighting, heating, and air conditioning.
3. Mechanical inspection: The mechanical components of the car are inspected for wear and tear. This includes checking the brakes, 

 llama3.2 sta scrivendo...: 218 token [00:16, 13.27 token/s]


Tempo per inferenza: 16.43 secondi

=D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D 
RISPOSTA GENERATA DA 'llama3.2':
=P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P 
Non ho la risposta specifica sulle operazioni di manutenzione per le ruote del treno ETR 500, poiché il contesto fornito non contiene informazioni su questo argomento. Tuttavia, posso dire che è comune che i controlli non distruttivi delle ruote del treno includano:

* Controlli di pressione e temperatura degli pneumi
* Verifiche dei bordi di usura dei dischi frenanti
* Controllo della consistenza e della presenza di cera sui dischi frenanti
* Verifiche della presenza di danni o deformazioni su supporti e perne delle ruote
* Controllo del funzionamento degli assi e dei loro componenti

Tuttavia, per una risposta più speci

 granite3-dense:2b sta scrivendo...: 868 token [00:52, 16.59 token/s]

Tempo per inferenza: 52.33 secondi

=D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D =D 
RISPOSTA GENERATA DA 'granite3-dense:2b':
=P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P =P 
Secondo il contesto fornito, i controlli non distruttivi previsti per le ruote del treno ETR 500 includono:

1. Pulse di isolamento: questi sono utilizzati per verificare la corretta funzionamento dell'isolamento delle ruote e per prevenire eventuali problemi causati da scarsissimo isolamento.
2. Controllo della pressione dell'olio dei ruoturi: questa controllo è fondamentale per garantire che gli olio dei ruotori siano correttamente lubrificati e che non ci siano deficit di lubrificazione.
3. Inspezione delle rubi: le rubi sono le parti che contattano il palloncino e la ruota del treno. Gli controlli di insiezio

In [8]:
df_risposte.to_csv('test_risposte_ETR500_ntop1.csv', index=False)